In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [3]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

In [4]:
os.environ['OPENAI_API_KEY'] = openai_api_key

In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

splits = text_splitter.split_documents(blog_docs)

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from langchain_core.prompts import ChatPromptTemplate

template = """
あなたはAI言語モデルアシスタントです。
あなたのタスクは、与えられたユーザーの質問に対して5つの異なるバージョンを生成し、ベクターデータベースから関連文書を検索することです。
ユーザーの質問に対して複数の視点を生成することで、距離ベースの類似性検索の限界をユーザーが克服できるように支援することが目標です。
これらの代替質問を改行で区切って入力してください。
元の質問: {question}
"""
prompt_perspective = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspective
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [8]:
from langchain_core.load import dumps, loads

def get_unique_union(documents: list[list]):
    flattend_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattend_docs))
    return [loads(doc) for doc in unique_docs]

question = "LLMエージェントにとってのタスク分解とはなにか？"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

/tmp/ipykernel_15703/1816738893.py:6: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


7

In [9]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """
このコンテキストに基づいて以下の質問に答えてください。

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
    "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'タスク分解とは、複雑なタスクを小さな管理しやすいサブゴールに分割することです。エージェントは大きなタスクを複数の小さなステップに分解し、それを実行するための計画を立てることが重要です。これにより、複雑なタスクを効率的に処理することが可能となります。'

In [10]:
template = """
あなたは、1つの入力クエリに基づいて複数の検索クエリを生成する便利なアシスタントです。\n
{question}に関連する複数の検索クエリを生成します。\n
出力（4クエリ）
"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [12]:
from langchain_core.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score) for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

6

In [13]:
from langchain_core.runnables import RunnablePassthrough

template = """
このコンテキストに基づいて以下の質問に答えてください。

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
    "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'LLMエージェントにとってのタスク分解とは、複雑なタスクをより小さな、管理しやすいサブゴールに分解することを指します。これにより、エージェントは複雑なタスクを効率的に処理することができます。'

In [14]:
template = """
あなたは、入力された質問に関連する複数のサブ質問を生成する、役に立つアシスタントです。\n
目標は、入力された質問を、単独で回答できるサブ問題/サブ質問のセットに分解することです。\n
{question} に関連する複数の検索クエリを生成します。\n
出力 (3 クエリ):
"""

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [15]:
llm = ChatOpenAI(temperature=0)

generate_queries_decomposition = (prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

question = "LLMのエージェントシステムのメインコンポーネントはなんですか？"
questions = generate_queries_decomposition.invoke({"question": question})

In [16]:
questions

['1. LLMのエージェントシステムのメインコンポーネントは何ですか？',
 '2. LLMのエージェントシステムにおけるメインコンポーネントの機能は何ですか？',
 '3. LLMのエージェントシステムにおいて、メインコンポーネントはどのようにして機能しますか？']

In [17]:
template = """
回答が必要な質問は次のとおりです。

\n --- \n {question} \n --- \n

利用可能な背景情報に関する質問と回答のペアは次のとおりです。

\n --- \n {q_a_pairs} \n --- \n

質問に関連する追加のコンテキストは次のとおりです。

\n --- \n {context} \n --- \n

上記のコンテキストと、背景情報に関する質問と回答のペアを使用して、質問に回答してください。\n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [18]:
def format_qa_pair(question, answer):
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

llm = ChatOpenAI(model_name="gpt-5.2", temperature=0)

q_a_pairs = ""
for q in questions:
    rag_chain = (
        {"context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "q_a_pairs": itemgetter("q_a_pairs")}
        | decomposition_prompt
        | llm
        | StrOutputParser()
    )
    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q, answer)
    q_a_pairs = q_a_pairs + "\n---\n" + q_a_pair

In [19]:
answer

'LLMを中核コントローラ（脳）とするエージェントシステムでは、LLM単体の推論・生成を **Planning（計画）** と **Memory（記憶）**（＋必要に応じて **Tool use（ツール利用）**）が補完し、次のように連携して機能します。\n\n- **Planning（計画）**  \n  - **サブゴール分解**：与えられた大きなタスクを、小さく実行可能な手順・目標に分割し、順序立てて進められる形にします。  \n  - **振り返り・改善（Reflection/Refinement）**：実行結果や過去の行動を自己批評・自己反省し、誤りや不足を学習して次の行動計画や最終出力を改善します（試行錯誤を前提に反復的に品質を上げる）。\n\n- **Memory（記憶）**  \n  - **短期記憶**：プロンプト内（インコンテキスト）にある直近の会話・状況を保持して、その場の推論に使います。  \n  - **長期記憶**：外部ストレージ（例：ベクターストア）に情報を蓄積し、必要時に検索・想起して、複数回のやり取りや長いタスクでも一貫性と継続性を保ちます。\n\n- **Tool use（ツール利用：補完的だが重要）**  \n  - LLMが外部APIや検索、コード実行などを呼び出して、モデル内部にない最新情報の取得や実世界への操作を行います。必要に応じて「APIが必要か→どのAPIか→入力を調整して再試行→結果を踏まえて回答」という反復も行います。\n\n要するに、**Planningが“何をどう進めるか”を組み立て、Memoryが“過去や必要知識”を供給し、Tool useが“外部で取得・実行”を担い**、LLMがそれらを統合して自律的にタスクを遂行します。'

In [20]:
from langchain_classic import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    sub_questions = sub_question_generator_chain.invoke({"question": question})
    rag_results = []
    for sub_question in sub_questions:
        retrieved_docs = retriever.invoke(sub_question)
        answer = (prompt_rag | llm | StrOutputParser()).invoke(
            {"context": retrieved_docs, "question": sub_question}
        )
        rag_results.append(answer)

    return rag_results, sub_questions

answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [21]:
def format_qa_pairs(questions, answers):
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

template = """
Q+A のペアを以下に示します。

{context}

これらを使用して、質問 {question} に対する回答を作成してください。
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context": context, "question": question})

'LLMのエージェントシステムのメインコンポーネントは、**LLMを中核コントローラ（脳）**として、それを補完する **Planning（計画）・Memory（記憶）・Tool use（ツール利用）** です。  \n特にPlanningには、**タスクのサブゴール分解**や**自己反省／改善（reflection/refinement）**が含まれます。'